# DQN agent with dietkit data

In [7]:
import numpy as np
import pandas as pd
from dqn_meal_planning_environment import MealPlanningEnv
from dietkit import (load_ingredient, load_menu, load_diet)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers.legacy import Adam

from rl.agents import DQNAgent  # NOTE: requires keras-rl2 not keras-rl
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [8]:
all_ingredients = load_ingredient(sample_language = 'eng')
meal_dict = load_menu(ingredients = all_ingredients, sample_language = 'eng')
example_mealplans = load_diet(menus = meal_dict, num_loads = 100, sample_language = 'eng', sample_name = 'ML')

possible_meals = np.array(list(meal_dict.keys()))
display(possible_meals)

nutrition_data = pd.DataFrame([meal.nutrition for meal in meal_dict.values()], index=possible_meals)
display(nutrition_data)

Exception ignored in: <function Menu.__del__ at 0x7f9fc05d8a60>
Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/rlclass/lib/python3.9/site-packages/dietkit/elements.py", line 89, in __del__
    del Menu.__catalog[self.name]
KeyError: 'Mushroom perilla seed soup'
Exception ignored in: <function Menu.__del__ at 0x7f9fc05d8a60>
Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/rlclass/lib/python3.9/site-packages/dietkit/elements.py", line 89, in __del__
    del Menu.__catalog[self.name]
KeyError: 'Zucchini soup'
Exception ignored in: <function Menu.__del__ at 0x7f9fc05d8a60>
Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/rlclass/lib/python3.9/site-packages/dietkit/elements.py", line 89, in __del__
    del Menu.__catalog[self.name]
KeyError: 'Potato zucchini soup'
Exception ignored in: <function Menu.__del__ at 0x7f9fc05d8a60>
Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/rlclass/lib/python3.9/site-package

array(['S watermelon punch (without milks)', 'Dried persimmon',
       'S kyogo grape (100g)', ..., 'Green onion kimchi',
       'Pickled paprika', 'empty'], dtype='<U72')

,Energy,Protein,Fat,Carbohydrate,Total Dietary,Calcium,Iron,Sodium,Vitamin A,Vitamin B1 (Thiamine),Vitamin B2 (Rivoflavin),Vitamin C,Linoleic Acid,Alpha-Linolenic Acid
S watermelon punch (without milks),55.00,0.7900,0.05000,13.8180,0.200,7.200,0.18600,0.600,71.083333,0.024000,0.03000,0.0000,14.2500,2.9300
Dried persimmon,76.50,0.5820,0.04800,20.5830,3.360,6.300,0.19200,0.900,13.725000,0.003000,0.06720,1.5330,0.0000,8.6010
S kyogo grape (100g),61.00,0.4300,0.04000,16.3800,0.400,4.000,0.17000,1.000,2.166667,0.026000,0.04700,0.1300,11.8900,6.0900
S dried persimmon,85.60,0.7720,0.03200,22.9800,3.400,5.600,0.24800,0.400,9.533333,0.003600,0.09720,1.0680,0.0000,5.0440
S fruit punch,101.05,3.6700,3.36450,14.8900,0.790,117.650,0.19850,36.000,91.166667,0.040350,0.19110,1.8925,86.6605,10.6925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ponytail radish kimchi,10.25,0.5050,0.05000,1.9725,0.775,10.500,0.16750,173.000,5.541667,0.003250,0.01900,0.4125,19.7050,9.4775
Kohlrabi watery kimchi,5.50,0.2900,0.01250,1.2900,0.600,10.500,0.06750,1.750,0.250000,0.203750,0.00875,0.0000,1.9650,5.1100
Green onion kimchi,12.25,0.6250,0.14500,2.1275,1.150,15.000,0.18250,163.750,26.854167,0.012750,0.02850,0.5550,79.8050,17.1425
Pickled paprika,7.94,0.2017,0.03005,1.9859,0.348,1.675,0.08145,67.204,4.850000,0.003675,0.02709,24.2820,12.4483,6.5448


In [9]:
num_meals = 21
env = MealPlanningEnv(possible_meals, nutrition_data, num_meals)

states = env.observation_space['nutrition_history'].shape
actions = env.action_space.n

def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=(1, *states)))
    model.add(Dense(24, activation='relu'))
    model.add(Flatten())
    model.add(Dense(actions, activation='linear'))
    return model

model = build_model(states, actions)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 1, 21, 24)         360       
                                                                 
 dense_7 (Dense)             (None, 1, 21, 24)         600       
                                                                 
 flatten_2 (Flatten)         (None, 504)               0         
                                                                 
 dense_8 (Dense)             (None, 3235)              1633675   
                                                                 
Total params: 1,634,635
Trainable params: 1,634,635
Non-trainable params: 0
_________________________________________________________________


In [10]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(
        model=model, 
        memory=memory, 
        policy=policy, 
        nb_actions=actions, 
        nb_steps_warmup=10, 
        target_model_update=1e-2
    )
    return dqn

dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=10000, visualize=False, verbose=1)

2023-04-17 17:51:55.020260: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_8/kernel/Assign' id:1050 op device:{requested: '', assigned: ''} def:{{{node dense_8/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_8/kernel, dense_8/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Training for 10000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 21:31 - reward: -22568.9261

/usr/local/anaconda3/envs/rlclass/lib/python3.9/site-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2023-04-17 17:51:55.372965: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_8/BiasAdd' id:1062 op device:{requested: '', assigned: ''} def:{{{node dense_8/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](dense_8/MatMul, dense_8/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-04-17 17:51:55.417021: W tensorflow/c/c_api.cc:300] Operation '{name:'count_6/Assign' id:1332 op device:{requested: '', assigned: ''} def:{{{node count_6/Assign}} = AssignVariableOp[_has_manual

10000/10000 [==============================] - 346s 35ms/step - reward: -12249.9590
done, took 345.511 seconds


In [12]:
results = dqn.test(env, nb_episodes=1, visualize=True)
print(np.mean(results.history['episode_reward']))

Testing for 1 episodes ...
Step: 1
Chosen Meal: Stir-fried seaweed stem and carrot
Meal History: ['Stir-fried seaweed stem and carrot' 'empty' 'empty' 'empty' 'empty'
 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty'
 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty']
Nutrition History: [[2.89600e+01 7.70000e-01 2.34000e+00 2.18000e+00 2.50000e-01 3.97000e+01
  1.70000e-01 2.05110e+02 3.19900e+01 1.00000e-02 4.00000e-02 3.94000e+00
  1.13877e+03 1.32990e+02]
 [0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00]
 [0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00]
 [0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e

None

Step: 2
Chosen Meal: Braised tofu and fried fish paste
Meal History: ['Stir-fried seaweed stem and carrot' 'Braised tofu and fried fish paste'
 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty'
 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty'
 'empty']
Nutrition History: [[2.89600e+01 7.70000e-01 2.34000e+00 2.18000e+00 2.50000e-01 3.97000e+01
  1.70000e-01 2.05110e+02 3.19900e+01 1.00000e-02 4.00000e-02 3.94000e+00
  1.13877e+03 1.32990e+02]
 [5.53800e+01 3.38000e+00 1.44000e+00 7.37000e+00 8.20000e-01 2.19400e+01
  4.50000e-01 3.01510e+02 3.84000e+00 1.00000e-02 8.00000e-02 1.00000e+00
  7.03020e+02 1.03580e+02]
 [0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00]
 [0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000

None

Step: 3
Chosen Meal: Deep-fried chicken without bones
Meal History: ['Stir-fried seaweed stem and carrot' 'Braised tofu and fried fish paste'
 'Deep-fried chicken without bones' 'empty' 'empty' 'empty' 'empty'
 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty'
 'empty' 'empty' 'empty' 'empty' 'empty']
Nutrition History: [[2.89600e+01 7.70000e-01 2.34000e+00 2.18000e+00 2.50000e-01 3.97000e+01
  1.70000e-01 2.05110e+02 3.19900e+01 1.00000e-02 4.00000e-02 3.94000e+00
  1.13877e+03 1.32990e+02]
 [5.53800e+01 3.38000e+00 1.44000e+00 7.37000e+00 8.20000e-01 2.19400e+01
  4.50000e-01 3.01510e+02 3.84000e+00 1.00000e-02 8.00000e-02 1.00000e+00
  7.03020e+02 1.03580e+02]
 [8.71000e+01 8.18000e+00 3.81000e+00 4.14000e+00 1.10000e-01 9.50000e+00
  4.60000e-01 6.77500e+01 2.09000e+01 7.00000e-02 9.00000e-02 0.00000e+00
  1.59808e+03 1.99310e+02]
 [0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000

...

Step: 20
Chosen Meal: Stir-fried seaweed stem and carrot
Meal History: ['Stir-fried seaweed stem and carrot' 'Braised tofu and fried fish paste'
 'Deep-fried chicken without bones' 'Braised tofu and fried fish paste'
 'Deep-fried chicken without bones' 'Deep-fried chicken without bones'
 'Deep-fried chicken without bones' 'Deep-fried chicken without bones'
 'Deep-fried chicken without bones' 'Pan-fried vegetables with mushroom'
 'Pan-fried vegetables with mushroom' 'Pan-fried vegetables with mushroom'
 'Pan-fried vegetables with mushroom' 'Pan-fried vegetables with mushroom'
 'Pan-fried vegetables with mushroom' 'Pan-fried vegetables with mushroom'
 'Stir-fried seaweed stem and carrot' 'Stir-fried seaweed stem and carrot'
 'Stir-fried seaweed stem and carrot' 'Stir-fried seaweed stem and carrot'
 'empty']
Nutrition History: [[2.89600e+01 7.70000e-01 2.34000e+00 2.18000e+00 2.50000e-01 3.97000e+01
  1.70000e-01 2.05110e+02 3.19900e+01 1.00000e-02 4.00000e-02 3.94000e+00
  1.13877e+03 1.

None

Step: 21
Chosen Meal: Stir-fried seaweed stem and carrot
Meal History: ['Stir-fried seaweed stem and carrot' 'Braised tofu and fried fish paste'
 'Deep-fried chicken without bones' 'Braised tofu and fried fish paste'
 'Deep-fried chicken without bones' 'Deep-fried chicken without bones'
 'Deep-fried chicken without bones' 'Deep-fried chicken without bones'
 'Deep-fried chicken without bones' 'Pan-fried vegetables with mushroom'
 'Pan-fried vegetables with mushroom' 'Pan-fried vegetables with mushroom'
 'Pan-fried vegetables with mushroom' 'Pan-fried vegetables with mushroom'
 'Pan-fried vegetables with mushroom' 'Pan-fried vegetables with mushroom'
 'Stir-fried seaweed stem and carrot' 'Stir-fried seaweed stem and carrot'
 'Stir-fried seaweed stem and carrot' 'Stir-fried seaweed stem and carrot'
 'Stir-fried seaweed stem and carrot']
Nutrition History: [[2.89600e+01 7.70000e-01 2.34000e+00 2.18000e+00 2.50000e-01 3.97000e+01
  1.70000e-01 2.05110e+02 3.19900e+01 1.00000e-02 4.00000e-02

None

Episode 1: reward: -241861.760, steps: 21
-241861.75983998057
